In [1]:
import pickle
import pandas as pd
import time
import json
import requests
import re
from lxml import html
from bs4 import BeautifulSoup
import csv
import tqdm
from urllib.parse import urljoin
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('start-maximized')
options.add_argument('window-size=1920x1080')  # Indstil en fast vinduesstørrelse

In [2]:
links = []  # Initialiserer en tom liste til at gemme de genererede links

# For-løkke, der itererer gennem værdierne
for offset in range(0, 330*10, 10):  
    # Genererer en URL med en specifik offset-værdi for hver iteration
    url = f'https://www.berlingske.dk/search?query=c25&limit=10&offset={offset}'
    links.append(url)  # Tilføjer den genererede URL til listen 'links'

links  # Viser listen med alle de genererede links

['https://www.berlingske.dk/search?query=c25&limit=10&offset=0',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=10',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=20',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=30',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=40',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=50',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=60',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=70',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=80',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=90',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=100',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=110',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=120',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=130',
 'https://www.berlingske.dk/search?query=c25&limit=10&offset=140',
 'http

In [3]:
base_url = 'https://www.berlingske.dk'
article_links = []

# Brug tqdm til at tracke tiden for hver iteration
for url in tqdm.tqdm(links):
    response = requests.get(url, 
                            headers={'name': 'Amanda Grønlund', 'email': 'cws883@alumni.ku.dk'})
    
    # Parse HTML med det samme uden at gemme i en liste
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find alle links (a-tags)
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Filter efter links, der indeholder "/virksomheder/" eller et andet mønster, du ønsker at matche
        if '/business/' in href or '/virksomheder/' in href or '/oekonomi/' in href or '/aktier/' in href or '/dinepenge/' in href or '/arbejdsliv/' in href or '/politik/' in href or '/indland/' in href or '/internationalt/' in href:  # Tilpas dette til det faktiske mønster af artikel-links
            full_url = urljoin(base_url, href)
            article_links.append(full_url)
    
    time.sleep(0.5)  # Vent 0,5 sekund for at undgå at overbelaste serveren

# Fjern eventuelle duplikater fra article_links
article_links = list(set(article_links))

100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [04:35<00:00,  1.20it/s]


In [4]:
article_links

['https://www.berlingske.dk/oekonomi/biotekboss-for-dansk-selskab-advarer-mod-usas-frygt-for-kina-ser-kaempe',
 'https://www.berlingske.dk/aktier/middagsaktier-coloplast-og-oersted-topper-i-stigende-eliteindeks',
 'https://www.berlingske.dk/aktier/torsdagens-aktier-medicinal-og-bankaktier-trak-c25-i-plus',
 'https://www.berlingske.dk/aktier/aktiestrateger-efter-aarets-hidtil-vaerste-dag-for-europaeiske-aktier-det',
 'https://www.berlingske.dk/aktier/mandagens-aktier-vestas-topper-sammen-med-coloplast-i-skriggroent-c25',
 'https://www.berlingske.dk/business/business-overblik-rentehop-og-prisstigninger-er-andres-nedtur-her-vaelter',
 'https://www.berlingske.dk/business/trods-bankuro-og-inflation-gode-maaneder-for-pensionsopsparinger',
 'https://www.berlingske.dk/aktier/torsdagens-aktier-dsv-og-udenlandske-regnskaber-vil-tage-fokus-i-roed-start',
 'https://www.berlingske.dk/aktier/royal-unibrew-hoejt-skum-i-aktiekursen',
 'https://www.berlingske.dk/business/torsdagens-aktier-jyske-bank-tr

In [5]:
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.implicitly_wait(15)  # Vent op til 10 sekunder på, at elementer bliver tilgængelige

# Dine login oplysninger
login = "jvgronlund@yahoo.com"
password = "rebklemmer72"

# Gå til Berlingskes login-side
driver.get('https://www.berlingske.dk/login')

# Afvis cookie-popup (hvis den er der)
try:
    accept_cookies_button = driver.find_element(By.CSS_SELECTOR, 'button[id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')
    accept_cookies_button.click()
    time.sleep(2)
except Exception as e:
    print("Ingen cookie-popup fundet, fortsætter uden.")

# Find login-felterne og indtast login-oplysninger
email_field = driver.find_element(By.ID, 'email')
password_field = driver.find_element(By.ID, 'current-password')

email_field.send_keys(login)
password_field.send_keys(password)

# Find og klik på login-knappen
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()

# Vent lidt for at sikre login-succes
time.sleep(5)

# Gå til Berlingske søgeresultater for C25
driver.get('https://www.berlingske.dk/search?query=c25')

# Vent lidt for at sikre, at siden er indlæst
time.sleep(5)

In [6]:
# Tom liste til at gemme artikeldetaljer
articles = []

# Loop gennem alle artikel-links
for link in tqdm.tqdm(article_links):
    try:
        # Brug Selenium driver til at navigere til siden
        driver.get(link)

        # Vent indtil siden er færdigindlæst og overskriften er tilgængelig
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
        
        # Scroll ned ad siden for at sikre, at alt indhold bliver indlæst
        time.sleep(2)  # Giv siden lidt tid til at indlæse
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Vent lidt for at sikre, at alt indlæses
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Hent overskriften (h2 er ofte brugt til hovedtitlen)
        title_element = driver.find_element(By.TAG_NAME, 'h2')
        title = title_element.text if title_element else 'No Title'
        
        # Hent datoen (tilpasset til den faktiske HTML-struktur på siden)
        date_element = driver.find_element(By.CLASS_NAME, 'article-byline__date')
        date = date_element.text if date_element else 'No Date'

        # Prøv først at hente hele artiklen ved at kombinere elementer fra forskellige sektioner
        try:
            intro_paragraph = driver.find_element(By.CSS_SELECTOR, 'p.article-header__intro').text
        except:
            intro_paragraph = ""

        # Prøv at hente teksten fra "articleBody"
        try:
            article_body = driver.find_element(By.ID, "articleBody")
            inner_html = article_body.get_attribute('innerHTML')
            soup = BeautifulSoup(inner_html, 'html.parser')
            paragraphs = soup.find_all('p')
            article_text = "\n".join([paragraph.get_text() for paragraph in paragraphs])
        except:
            article_text = ""

        # Kombinér intro-tekst og artikel-tekst fra "articleBody"
        full_article_text = f"{intro_paragraph}\n{article_text}" if intro_paragraph else article_text

        # Hvis teksten stadig er mangelfuld, prøv at finde alternative sektioner (som i HTML1)
        if not full_article_text or len(full_article_text) < 100:  # Brug en passende tærskel
            alternative_sections = driver.find_elements(By.CSS_SELECTOR, 'div.article__section p')
            if alternative_sections:
                alternative_text = "\n".join([section.text for section in alternative_sections])
                if alternative_text:
                    full_article_text = f"{intro_paragraph}\n{alternative_text}" if intro_paragraph else alternative_text
        
        # Tilføj detaljerne til listen
        articles.append({
            'URL': link,
            'Title': title,
            'Date': date,
            'Content': full_article_text
        })

    except Exception as e:
        print(f"An error occurred for {link}: {e}")

# Luk driveren, når alle sider er blevet besøgt
driver.quit()

# Konverter listen af artikeldetaljer til en pandas DataFrame
df = pd.DataFrame(articles)

  1%|▌                                                                            | 20/3080 [08:24<81:03:55, 95.37s/it]

An error occurred for https://www.berlingske.dk/aktier/fredagens-aktier-vestas-kravlede-op-i-rekordhoejt-c25-indeks: Message: timeout: Timed out receiving message from renderer: 299.601
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x01198923+23283]
	(No symbol) [0x0115E934]
	(No symbol) [0x01090733]
	(No symbol) [0x0107C90D]
	(No symbol) [0x0107C778]
	(No symbol) [0x0107AEDD]
	(No symbol) [0x0107B31D]
	(No symbol) [0x010873FA]
	(No symbol) [0x0109B6E1]
	(No symbol) [0x0109FC16]
	(No symbol) [0x0107B94C]
	(No symbol) [0x0109B49D]
	(No symbol) [0x0110CE37]
	(No symbol) [0x010F41B6]
	(No symbol) [0x010C8017]
	(No symbol) [0x010C890D]
	GetHandleVerifier [0x0128A5F3+1013699]
	GetHandleVerifier [0x01293E4C+1052700]
	GetHandleVerifier [0x0128D4B4+1025668]
	GetHandleVerifier [0x011BEA2B+179195]
	(No symbol) [0x01166833]
	(No symbol) [0x01163198]
	(No symbol) [0x01163337]
	(No symbol) [0x0115B4BE]
	BaseThreadInitThunk [0x770A7BA9+25]
	RtlInitializeExceptionChain [0x77

  5%|███▊                                                                        | 153/3080 [30:46<77:52:55, 95.79s/it]

An error occurred for https://www.berlingske.dk/aktier/tirsdagens-aktier-genmab-til-tops-i-paent-stigende-eliteindeks: Message: timeout: Timed out receiving message from renderer: 299.201
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x01198923+23283]
	(No symbol) [0x0115E934]
	(No symbol) [0x01090733]
	(No symbol) [0x0107C90D]
	(No symbol) [0x0107C778]
	(No symbol) [0x0107AEDD]
	(No symbol) [0x0107B31D]
	(No symbol) [0x010873FA]
	(No symbol) [0x0109B6E1]
	(No symbol) [0x0109FC16]
	(No symbol) [0x0107B94C]
	(No symbol) [0x0109B49D]
	(No symbol) [0x0110CE37]
	(No symbol) [0x010F41B6]
	(No symbol) [0x010C8017]
	(No symbol) [0x010C890D]
	GetHandleVerifier [0x0128A5F3+1013699]
	GetHandleVerifier [0x01293E4C+1052700]
	GetHandleVerifier [0x0128D4B4+1025668]
	GetHandleVerifier [0x011BEA2B+179195]
	(No symbol) [0x01166833]
	(No symbol) [0x01163198]
	(No symbol) [0x01163337]
	(No symbol) [0x0115B4BE]
	BaseThreadInitThunk [0x770A7BA9+25]
	RtlInitializeExceptionChain [0x

 18%|█████████████▎                                                            | 555/3080 [1:27:47<67:14:46, 95.88s/it]

...

 40%|████████████████████████████▉                                           | 1240/3080 [4:29:43<72:55:06, 142.67s/it]

An error occurred for https://www.berlingske.dk/aktier/renten-dansede-paa-stedet-i-stille-marked: Message: timeout: Timed out receiving message from renderer: 299.628
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x01198923+23283]
	(No symbol) [0x0115E934]
	(No symbol) [0x01090733]
	(No symbol) [0x0107C90D]
	(No symbol) [0x0107C778]
	(No symbol) [0x0107AEDD]
	(No symbol) [0x0107B31D]
	(No symbol) [0x010873FA]
	(No symbol) [0x0109B6E1]
	(No symbol) [0x0109FC16]
	(No symbol) [0x0107B94C]
	(No symbol) [0x0109B49D]
	(No symbol) [0x0110CE37]
	(No symbol) [0x010F41B6]
	(No symbol) [0x010C8017]
	(No symbol) [0x010C890D]
	GetHandleVerifier [0x0128A5F3+1013699]
	GetHandleVerifier [0x01293E4C+1052700]
	GetHandleVerifier [0x0128D4B4+1025668]
	GetHandleVerifier [0x011BEA2B+179195]
	(No symbol) [0x01166833]
	(No symbol) [0x01163198]
	(No symbol) [0x01163337]
	(No symbol) [0x0115B4BE]
	BaseThreadInitThunk [0x770A7BA9+25]
	RtlInitializeExceptionChain [0x7788C10B+107]
	RtlCle

 96%|█████████████████████████████████████████████████████████████████████████▏  | 2965/3080 [7:56:47<27:03, 14.12s/it]

An error occurred for https://www.radio24syv.dk/aktier/millionaerklubben-12-01-2018: Message: 
Stacktrace:
	GetHandleVerifier [0x01198923+23283]
	(No symbol) [0x0115E934]
	(No symbol) [0x01090733]
	(No symbol) [0x010D326F]
	(No symbol) [0x010D34AB]
	(No symbol) [0x0110EE42]
	(No symbol) [0x010F4464]
	(No symbol) [0x0110CB8D]
	(No symbol) [0x010F41B6]
	(No symbol) [0x010C8017]
	(No symbol) [0x010C890D]
	GetHandleVerifier [0x0128A5F3+1013699]
	GetHandleVerifier [0x01293E4C+1052700]
	GetHandleVerifier [0x0128D4B4+1025668]
	GetHandleVerifier [0x011BEA2B+179195]
	(No symbol) [0x01166833]
	(No symbol) [0x01163198]
	(No symbol) [0x01163337]
	(No symbol) [0x0115B4BE]
	BaseThreadInitThunk [0x770A7BA9+25]
	RtlInitializeExceptionChain [0x7788C10B+107]
	RtlClearBits [0x7788C08F+191]



100%|████████████████████████████████████████████████████████████████████████████| 3080/3080 [8:09:54<00:00,  9.54s/it]


In [7]:
# Juster Pandas-indstillinger for at vise hele indholdet i 'Content' kolonnen
pd.set_option('display.max_colwidth', None)

# Vis DataFrame
df

URL  \
0       https://www.berlingske.dk/oekonomi/biotekboss-for-dansk-selskab-advarer-mod-usas-frygt-for-kina-ser-kaempe   
1                https://www.berlingske.dk/aktier/middagsaktier-coloplast-og-oersted-topper-i-stigende-eliteindeks   
2                       https://www.berlingske.dk/aktier/torsdagens-aktier-medicinal-og-bankaktier-trak-c25-i-plus   
3       https://www.berlingske.dk/aktier/aktiestrateger-efter-aarets-hidtil-vaerste-dag-for-europaeiske-aktier-det   
4           https://www.berlingske.dk/aktier/mandagens-aktier-vestas-topper-sammen-med-coloplast-i-skriggroent-c25   
...                                                                                                            ...   
3053  https://www.berlingske.dk/virksomheder/droemmer-du-om-at-faa-et-af-danmarks-mest-magtfulde-job-her-er-det-du   
3054    https://www.berlingske.dk/aktier/fredagens-aktier-markedet-ignorerede-toldstrid-og-sendte-william-demant-i   
3055          https://www.berlingske.dk/aktier/mandagens-aktier-novo-topper-fladt-c25-og-sas-er-under-fortsat-pres   
3056            https://www.berlingske.dk/virksomheder/analytikere-havde-haabet-paa-flere-solgte-iphones-fra-apple   
3057                                 https://www.berlingske.dk/aktier/novo-aktien-i-frit-fald-traekker-c25-i-roedt   

                                                                                                                     Title  \
0                                      Biotekboss for dansk selskab advarer mod USAs frygt for Kina – ser kæmpe potentiale   
1                                                         Middagsaktier: Coloplast og Ørsted topper i stigende eliteindeks   
2                                                              Torsdagens aktier: Medicinal- og bankaktier trak C25 i plus   
3     Aktiestrateger efter årets hidtil værste dag for europæiske aktier: Det er inflationsspøgelset, der sætter retningen   
4                                                    Mandagens aktier: Vestas topper sammen med Coloplast i skriggrønt C25   
...                                                                                                                    ...   
3053                                      Drømmer du om at få et af Danmarks mest magtfulde job? Her er det, du skal kunne   
3054                                     Fredagens aktier: Markedet ignorerede toldstrid og sendte William Demant i rekord   
3055                                                  Mandagens aktier: Novo topper fladt C25 og SAS er under fortsat pres   
3056                                                             Analytikere havde håbet på flere solgte iPhones fra Apple   
3057                                                                          Novo-aktien i frit fald – trækker C25 i rødt   

                                          Date  \
0           Tirsdag d. 14. maj 2024, kl. 15.09   
1      Tirsdag d. 03. december 2019, kl. 12.12   
2     Torsdag d. 19. september 2019, kl. 17.05   
3            Onsdag d. 12. maj 2021, kl. 13.52   
4           Mandag d. 01. juli 2019, kl. 09.16   
...                                        ...   
3053     Mandag d. 14. februar 2022, kl. 18.25   
3054        Fredag d. 06. juli 2018, kl. 17.06   
3055       Mandag d. 29. april 2019, kl. 11.49   
3056         Onsdag d. 02. maj 2018, kl. 00.00   
3057    Mandag d. 20. december 2021, kl. 10.25   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [8]:
# Gem DataFrame til en CSV-fil, hvis ønsket
df.to_csv('berlingske_c25_Endelig.csv', index=False)